Limpeza e Filtragem
O Problema: Temos gigabytes de dados misturados (Renda Fixa, Cambial, Multimercado). O desafio pede especificamente a "classe de Ações".

Script:

1 - Ler o arquivo de Cadastro (cad_fi.csv) para pegar os CNPJs que são de "Ações".
2 - Ler os arquivos mensais (inf_diario...) um por um.
3 - Filtrar apenas as linhas que têm esses CNPJs.
4 - Salvar tudo num arquivo único e otimizado (formato Parquet, que é muito mais rápido que CSV para leitura posterior).

Importamos as bibliotecas e definimos os caminhos. Usaremos o cad_fi.csv para filtrar os fundos de interesse antes de processar as séries temporais pesadas.

## 1. Imports e Configuração
Importamos as bibliotecas e definimos os caminhos. Usaremos o cad_fi.csv para filtrar os fundos de interesse antes de processar as séries temporais pesadas.

## 2. Filtrando os CNPJs de Ações

Segundo o escopo do desafio, devemos utilizar a classe de Ações. Carregamos o cadastro (cad_fi.csv), filtramos por CLASSE == 'Fundo de Ações' e status EM FUNCIONAMENTO NORMAL. Isso gera uma lista de CNPJs válidos ("White List") para aplicarmos nos dados diários.

## 3. Loop de Processamento (O "Triturador")

Agora iteramos sobre todos os arquivos mensais baixados. Para cada arquivo:

1 - Carregamos o CSV na memória.
2 - Filtramos mantendo apenas os CNPJs de Ações identificados no passo anterior.
3 - Acumulamos o resultado.

Otimização: Usamos gc.collect() para liberar memória RAM entre as iterações, evitando que o computador trave com o volume de dados.

## 4. Salvando em Parquet (CORRIGIR)


Salvamos o resultado consolidado em formato Parquet. (corrigir)
Por que Parquet? Ocupa ~80% menos espaço em disco que CSV.
Mantém os tipos de dados (datas, números) corretos, evitando ter que converter tudo de novo na próxima etapa.

In [21]:
import pandas as pd
import glob
import os
import gc

# ==============================================================================
# 1. CONFIGURAÇÃO
# ==============================================================================
RAW_DIR = '../data/raw'
PROCESSED_DIR = '../data/processed'
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Lista dos arquivos
arquivos_diarios = sorted(glob.glob(f'{RAW_DIR}/inf_diario_fi_*.csv'))
colunas_importantes = ['CNPJ_FUNDO', 'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'CAPTC_DIA', 'RESG_DIA', 'NR_COTST']

print(f"🚀 Iniciando Processamento (Modo CSV)...")

# ==============================================================================
# 2. FILTRO DE AÇÕES (WHITE LIST)
# ==============================================================================
print("📂 Lendo Cadastro...")
try:
    df_cad = pd.read_csv(f'{RAW_DIR}/cad_fi.csv', sep=';', encoding='latin1', low_memory=False)
    df_cad.columns = df_cad.columns.str.upper().str.strip()

    # Filtro Corrigido
    filtro_acoes = df_cad['CLASSE'].str.strip() == 'Ações'
    filtro_ativo = df_cad['SIT'] == 'EM FUNCIONAMENTO NORMAL'
    cnpjs_acoes = set(df_cad[filtro_acoes & filtro_ativo]['CNPJ_FUNDO'].unique())

    del df_cad
    gc.collect()
except Exception as e:
    print(f"Erro no cadastro: {e}")

# ==============================================================================
# 3. LEITURA DOS ARQUIVOS
# ==============================================================================
dfs_filtrados = []
print(f"🔄 Lendo {len(arquivos_diarios)} arquivos...")

for arquivo in arquivos_diarios:
    try:
        df_mes = pd.read_csv(arquivo, sep=';', encoding='latin1', low_memory=False)
        df_mes.columns = df_mes.columns.str.upper().str.strip()

        # Correção Resolução 175
        if 'CNPJ_FUNDO_CLASSE' in df_mes.columns and 'CNPJ_FUNDO' not in df_mes.columns:
            df_mes.rename(columns={'CNPJ_FUNDO_CLASSE': 'CNPJ_FUNDO'}, inplace=True)

        if 'CNPJ_FUNDO' in df_mes.columns:
            df_mes = df_mes[df_mes['CNPJ_FUNDO'].isin(cnpjs_acoes)]
            if not df_mes.empty:
                cols = [c for c in colunas_importantes if c in df_mes.columns]
                dfs_filtrados.append(df_mes[cols])

        del df_mes
        gc.collect()
    except:
        pass

# ==============================================================================
# 4. SALVAMENTO EM CSV (A SOLUÇÃO DEFINITIVA)
# ==============================================================================
if len(dfs_filtrados) > 0:
    print("🧩 Concatenando...")
    df_final = pd.concat(dfs_filtrados, ignore_index=True)

    print("📅 Ajustando Datas...")
    df_final['DT_COMPTC'] = pd.to_datetime(df_final['DT_COMPTC'])
    df_final.sort_values(by=['CNPJ_FUNDO', 'DT_COMPTC'], inplace=True)

    # --- MUDANÇA AQUI: CSV EM VEZ DE PARQUET ---
    arquivo_saida = f'{PROCESSED_DIR}/base_acoes_consolidada.csv'

    print(f"💾 Salvando em CSV: {arquivo_saida}")
    df_final.to_csv(arquivo_saida, index=False, sep=';')

    print("-" * 30)
    print(f"✅ SUCESSO! Arquivo CSV salvo.")
    print("-" * 30)
else:
    print("💀 Erro: Lista vazia.")

🚀 Iniciando Processamento (Modo CSV)...
📂 Lendo Cadastro...
🔄 Lendo 25 arquivos...
🧩 Concatenando...
📅 Ajustando Datas...
💾 Salvando em CSV: ../data/processed/base_acoes_consolidada.csv
------------------------------
✅ SUCESSO! Arquivo CSV salvo.
------------------------------
